In [ ]:
#install visdom
!pip install visdom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=d54f30aa0fb43b75aa13ed1695d00d0dceb18208c216e459562205365545bd96
  Stored in directory: /root/.cache/pip/wheels/37/6c/38/64eeaa310e325aacda723e6df1f79ab5e9f31ba195264e04a8
Successfully built visdom


In [ ]:
# start visdom server in background
import subprocess
subprocess.Popen(["python", "-m", "visdom.server", "-port", "8097"])
import time; time.sleep(3)

In [ ]:
# user interface visualization with grok
!pip install pyngrok
from pyngrok import ngrok


import subprocess
import time

# start visdom server in the background
subprocess.Popen(["python", "-m", "visdom.server", "-port", "8097"])
time.sleep(3)  # give it a few seconds to boot up

# then connect ngrok
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("36QylUhTI8bXFJ5rOZskDze1ka6_qw5dGXXyaeB4bjvvvSmS")
url = ngrok.connect(8097)
print(f"open visdom at: {url}")

open visdom at: NgrokTunnel: "https://unsatisfiable-yair-satiably.ngrok-free.dev" -> "http://localhost:8097"


In [ ]:
# alternative cell to run if grok doesnt start, there will be no visdom user interface
from visdom import Visdom
viz = Visdom(port=8097)
print("connected to visdom:", viz.check_connection())

Connected to Visdom: True


1. Train a simple CNN on MNIST with Visdom logging (manual)

In [ ]:
# standard imports ML and NN related
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
# import visdom for data visualizations
from visdom import Visdom

# simple CNN to build for task 1, with 2 convolutional layer, 1 fully connected layer and 1 output layer:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
# forward pass defining the how dataas flows along the network:
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# connect to a visdom server on port 8097
viz = Visdom(port=8097)
# gpu if avaiable othervise cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# move the weight to the device
model = SimpleCNN().to(device)
# chosing the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# vhoding the loss function
criterion = nn.CrossEntropyLoss()

# normalization for stable training
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
# load the training set (in batches)
train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform), batch_size=64, shuffle=True)
# load the evaluation set (not need to be in batches)
test_loader = DataLoader(datasets.MNIST('./data', train=False, transform=transform), batch_size=1000)

# train for full 10 full epochs
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device) # move data and target to the cpu or the gpu
        optimizer.zero_grad() # clear gradient from previos batch
        loss = criterion(model(data), target) # compute the loss beetween the data that made trough a forward pass and the target
        loss.backward() # backpropagation
        optimizer.step() # weights update
        running_loss += loss.item() # loss accumulation

    avg_loss = running_loss / len(train_loader) # average loss acroass the batches
    # as asked by task 1 I do a visdom manual logging.
    # win='loss' identifies which plot window to update. update='append' adds to the existing line rather than replacing it. The opts dict just sets titles and labels.
    viz.line(Y=[avg_loss], X=[epoch], win='loss', update='append',
             opts=dict(title='training loss', xlabel='epoch', ylabel='loss'))

    # switch to evaluation
    model.eval()
    correct = 0
    # I disable gradient here because I dont need it here
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            correct += model(data).argmax(1).eq(target).sum().item()
    acc = correct / len(test_loader.dataset)
    # ANOTHER visdom manual logging for accuracy...
    viz.line(Y=[acc], X=[epoch], win='acc', update='append',
             opts=dict(title='accuracy', xlabel='epoch', ylabel='accuracy'))
    # and ANOTHER visdom manual logging for learning rate...
    viz.line(Y=[optimizer.param_groups[0]['lr']], X=[epoch], win='lr', update='append',
             opts=dict(title='learning rate', xlabel='epoch', ylabel='LR'))

    print(f"epoch {epoch}: loss={avg_loss:.4f}, acc={acc:.4f}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 19.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 523kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.83MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.79MB/s]


epoch 0: loss=0.1163, acc=0.9864
epoch 1: loss=0.0366, acc=0.9862
epoch 2: loss=0.0226, acc=0.9892
epoch 3: loss=0.0146, acc=0.9889
epoch 4: loss=0.0124, acc=0.9902
epoch 5: loss=0.0083, acc=0.9899
epoch 6: loss=0.0084, acc=0.9887
epoch 7: loss=0.0052, acc=0.9901
epoch 8: loss=0.0067, acc=0.9871
epoch 9: loss=0.0054, acc=0.9866


 2. Install PyTorch Lightning and understand logger interface                                                                                                                                    

In [ ]:
# install pytorch lightning as asked by task 2, this would help to handle the boilerplater discussed in the google docs
!pip install lightning -q

import lightning.pytorch as pl
from lightning.pytorch.loggers import Logger # it's a blueprint that defines what methods a logger must have
import inspect # its important to inspect what is inside other class, specifically in Logger

# inspect all the method inside the Logger, this is part of understand the logger interface for task 2
for name, method in inspect.getmembers(Logger, predicate=inspect.isfunction):
    if not name.startswith('_'):
        sig = inspect.signature(method)
        print(f"{name}{sig}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 45.6 MB/s eta 0:00:00
after_save_checkpoint(self, checkpoint_callback: lightning.pytorch.callbacks.model_checkpoint.ModelCheckpoint) -> None
finalize(self, status: str) -> None
log_graph(self, model: torch.nn.modules.module.Module, input_array: Optional[torch.Tensor] = None) -> None
log_hyperparams(self, params: Union[dict[str, Any], argparse.Namespace], *args: Any, **kwargs: Any) -> None
log_metrics(self, metrics: dict[str, float], step: Optional[int] = None) -> None
save(self) -> None


 3. Implement a basic custom Lightning logger   

In [ ]:
# after the exploration of Logger lets try to do task 3 and implement a basic custom lightning logger
from lightning.pytorch.loggers import Logger
from visdom import Visdom
from argparse import Namespace

# custom lightning logger:
class VisdomLogger(Logger):
    def __init__(self, env="main", port=8097):
        super().__init__()
        self._viz = Visdom(port=port, env=env)
        self._env = env

# from the output of the previous cell I understand that Lightning requires every logger to have a name...
    @property
    def name(self):
        return "VisdomLogger"
# ...and a version:
    @property
    def version(self):
        return "0.1"

    # function for the logging of the model settings (loss, accuracy and learning rate)
    def log_hyperparams(self, params, *args, **kwargs):
        if isinstance(params, Namespace):
            params = vars(params)
        text = "<br>".join(f"<b>{k}</b>: {v}" for k, v in params.items())
        self._viz.text(text, win="hyperparams", opts=dict(title="Hyperparameters"))

    # eveytime a metric get logged this runs and plot each one on visdom automatically
    # notice how the viz.line command of two cells ago it inside a for now, so it get repeated
    def log_metrics(self, metrics, step=None):
        for key, value in metrics.items():
            self._viz.line(
                Y=[float(value)], X=[step or 0],
                win=key, update='append',
                opts=dict(title=key, xlabel='step', ylabel=key)
            )


# wrapping the model in a lightning module
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = SimpleCNN()  # reuse the same CNN from task 1
        self.criterion = nn.CrossEntropyLoss()

    # as task 1 we defined how data flows through
    def forward(self, x):
        return self.model(x)

    # lightning handle for each batch zero_grad(), backward(), optimizer.step()
    def training_step(self, batch, batch_idx):
        x, y = batch
        loss = self.criterion(self(x), y)
        self.log("train_loss", loss)
        return loss

    # as task 1 we definte the validation_step
    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x).argmax(1)
        acc = pred.eq(y).float().mean()
        self.log("val_acc", acc)  # one line, no viz.line() boilerplate

    # as task 1 we define the optimizer
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)

# create the logger, create the trainer and hand it to the logger
logger = VisdomLogger(port=8097)
trainer = pl.Trainer(max_epochs=5, logger=logger)
trainer.fit(
    MNISTModel(),
    train_dataloaders=train_loader,
    val_dataloaders=test_loader
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: 💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch

┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name      ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model     │ SimpleCNN        │  1.2 M │ train │     0 │
│ 1 │ criterion │ CrossEntropyLoss │      0 │ train │     0 │
└───┴───────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 1.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.2 M                                                                                                
Total estimated model params size (MB): 4                                                                          
Modules in train mode: 6                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)`
is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


 4. Profile hook overhead in PyTorch                                                                                                                                                                                 


In [ ]:
# will our implementation with lightning for solving the boilerplater slow the training process to much? let's find out
import torch
import torch.nn as nn
import time

# we consider a big and deep model for make noticible the eventual slow down of implementation with lightning
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(*[nn.Linear(512, 512) for _ in range(20)])

    def forward(self, x):
        return self.layers(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BigModel().to(device)
x = torch.randn(64, 512).to(device) # fake input data, I just need to feed something to the model, I am interested not in the output but to the benchmarks

# warmup, I run the model 50 times before measuring anything bcyase the GPUs are lazy
for _ in range(50):
    out = model(x)
    out.sum().backward()
torch.cuda.synchronize()

# benchmark WITHOUT hooks, run 500 forward and backward passes and time how long it takes
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(500):
    out = model(x)
    out.sum().backward()
torch.cuda.synchronize()
baseline = time.perf_counter() - start

handles = [] # store references to every hook
hook_data = {} # dictionary where hooks will dump their data

# loops through every layer in the model, name is the label, module is the actual layer object itself
for name, module in model.named_modules():
    def fwd_hook(mod, inp, out, n=name):
        hook_data[n] = out.detach().norm().item() # save the norm (.norm()) of the output of the layer a plain pytoch number (.item())
    def bwd_hook(mod, grad_in, grad_out, n=name):
        if grad_out[0] is not None:
            hook_data[f"{n}_grad"] = grad_out[0].detach().norm().item() # save the norm (.norm()) of the gradient (if produced) of the layer a plain pytoch number (.item())

    # attaches both hooks to the layer
    handles.append(module.register_forward_hook(fwd_hook))
    handles.append(module.register_full_backward_hook(bwd_hook))

# benchmark WITH hooks, run 500 forward and backward passes and time how long it takes
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(500):
    out = model(x)
    out.sum().backward()
torch.cuda.synchronize()
hooked = time.perf_counter() - start

# remove all hooks, leaving hooks means they keep running forever and slowing everything else down:
for h in handles:
    h.remove()

# calculate percentage slowdown:
perc_slodown = ((hooked - baseline) / baseline) * 100
print(f"baseline:   {baseline:.3f}s")
print(f"with hooks: {hooked:.3f}s")
print(f"percentage slowdown:   {perc_slodown:.1f}%")

sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


baseline:   2.952s
with hooks: 8.904s
percentage slowdown:   201.6%


the percentage slowdown is big, way to big. the culprit is probably .item that forces a GPU->CPU syncronzation on very single hook call

In [ ]:
# one little difference with respect to the code from before is in no .item in fwd_hook() and bwd_hook()
handles = []
hook_data = {}
for name, module in model.named_modules():
    def fwd_hook(mod, inp, out, n=name):
        hook_data[n] = out.detach().norm()  # stays on GPU, no sync
    def bwd_hook(mod, grad_in, grad_out, n=name):
        if grad_out[0] is not None:
            hook_data[f"{n}_grad"] = grad_out[0].detach().norm()  # no .item()
    handles.append(module.register_forward_hook(fwd_hook))
    handles.append(module.register_full_backward_hook(bwd_hook))

# Benchmark optimized hooks
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(500):
    out = model(x)
    out.sum().backward()
torch.cuda.synchronize()
hooked_v2 = time.perf_counter() - start

for h in handles:
    h.remove()

perc_slodown_v2 = ((hooked_v2 - baseline) / baseline) * 100
print(f"baseline:          {baseline:.3f}s")
print(f"hooks (.item()):   {hooked:.3f}s  → {162.2:.1f}% overhead")
print(f"hooks (no .item()): {hooked_v2:.3f}s  → {perc_slodown_v2:.1f}% overhead")

baseline:          2.952s
hooks (.item()):   8.904s  → 162.2% overhead
hooks (no .item()): 4.200s  → 42.3% overhead


still high, probably the .norm() computation itself is adding to much computation too, it launches a gpu kernel in every hook. Let's try 3 experiments to figure out where the slowdown actually comes from.

In [ ]:
# test 1: let's measure the slowdown of just having empty hooks, fwd_hook() and bwd_hook() have just a pass, means empty
handles = []
for name, module in model.named_modules():
    def fwd_hook(mod, inp, out):
        pass
    def bwd_hook(mod, grad_in, grad_out):
        pass
    handles.append(module.register_forward_hook(fwd_hook))
    handles.append(module.register_full_backward_hook(bwd_hook))

torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(500):
    out = model(x)
    out.sum().backward()
torch.cuda.synchronize()
empty_hooks = time.perf_counter() - start
for h in handles:
    h.remove()

# test 2: let's measure the slowdown of giving things to the hooks, but we dont have computations here so no .norm()
handles = []
hook_data = {}
for name, module in model.named_modules():
    def fwd_hook(mod, inp, out, n=name):
        hook_data[n] = out.detach()  # just store, no norm
    def bwd_hook(mod, grad_in, grad_out, n=name):
        if grad_out[0] is not None:
            hook_data[f"{n}_grad"] = grad_out[0].detach()
    handles.append(module.register_forward_hook(fwd_hook))
    handles.append(module.register_full_backward_hook(bwd_hook))

torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(500):
    out = model(x)
    out.sum().backward()
torch.cuda.synchronize()
store_hooks = time.perf_counter() - start
for h in handles:
    h.remove()

# test 3: let's measure now the slowdown of the compotutations, but in a realistic case,
# so in the case in which we compute norm only every 100 steps: in fact, when we
# monitoring, we dont need to log every single step
handles = []
hook_data = {}
step = [0]
for name, module in model.named_modules():
    def fwd_hook(mod, inp, out, n=name):
        if step[0] % 100 == 0:
            hook_data[n] = out.detach().norm()
    def bwd_hook(mod, grad_in, grad_out, n=name):
        if step[0] % 100 == 0 and grad_out[0] is not None:
            hook_data[f"{n}_grad"] = grad_out[0].detach().norm()
    handles.append(module.register_forward_hook(fwd_hook))
    handles.append(module.register_full_backward_hook(bwd_hook))

torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(500):
    out = model(x)
    out.sum().backward()
    step[0] += 1
torch.cuda.synchronize()
sampled_hooks = time.perf_counter() - start
for h in handles:
    h.remove()

print(f"baseline:              {baseline:.3f}s")
print(f"slowdown with empty hooks:           {empty_hooks:.3f}s  → {((empty_hooks-baseline)/baseline)*100:.1f}%")
print(f"slowdown with store ref only:        {store_hooks:.3f}s  → {((store_hooks-baseline)/baseline)*100:.1f}%")
print(f"slowdown with norm every 100 steps:  {sampled_hooks:.3f}s  → {((sampled_hooks-baseline)/baseline)*100:.1f}%")

baseline:              2.952s
slowdown with empty hooks:           3.371s  → 14.2%
slowdown with store ref only:        3.854s  → 30.5%
slowdown with norm every 100 steps:  3.099s  → 5.0%


the biggest slowdown increase was caused by empty hooks, this is slowdown is high because the model is small and fast, on a real model with heavier layers that ratio shrinks drammatically so we are good.

 5. Complete starter task: "Add a gradient norm logger to Visdom client"       

In [ ]:
from visdom import Visdom
import torch

# define the gradient norm logger
class GradientNormLogger:
    def __init__(self, model, viz=None, port=8097, log_every=50):
        self.viz = viz or Visdom(port=port) # viz connection
        self.model = model
        self.step = 0 # tracking the trainining steps
        self.log_every = log_every # log every tot step not every single one
        self._handles = [] # stores hook references
        self._attach(model) # install hooks

    def _attach(self, model):
        # loop through every individual weight/bias in the model
        for name, param in model.named_parameters():
            # skip frozen/non-trainable params:
            if param.requires_grad:
                def hook(grad, layer_name=name):
                    if self.step % self.log_every == 0: # only do work every tot steps
                        norm = grad.detach().norm(2).item() # L2 norm will summarize how much large is this gradient
                        # plot it on visdom:
                        self.viz.line(
                            Y=[norm], X=[self.step],
                            win=f"grad_norm/{layer_name}",
                            update='append',
                            opts=dict(title=f"grad norm: {layer_name}",
                                      xlabel="step", ylabel="L2 norm")
                        )
                # actually install the hook on this parameter
                self._handles.append(param.register_hook(hook))

    def on_step(self):
        if self.step % self.log_every == 0:
            total_norm = torch.nn.utils.clip_grad_norm_(
                self.model.parameters(), float('inf'))
            self.viz.line(
                Y=[total_norm.item()], X=[self.step],
                win="grad_norm/total", update='append',
                opts=dict(title="total gradient norm",
                          xlabel="step", ylabel="L2 norm")
            )
        self.step += 1

    def detach(self):
        # remove all hooks, leaving hooks means they keep running forever and slowing everything else down:
        for h in self._handles:
            h.remove()
        self._handles.clear()


# test it with a quick training run
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# installs all the hooks on the model
grad_logger = GradientNormLogger(model, log_every=50)

for epoch in range(2):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        loss = criterion(model(data), target)
        loss.backward()
        optimizer.step()
        grad_logger.on_step()
    print(f"Epoch {epoch}: loss={loss.item():.4f}")

# always clean up hooks when done
grad_logger.detach()
print("ok, all gradient norms were logged to visdom")

Epoch 0: loss=0.0047
Epoch 1: loss=0.0281
ok, all gradient norms were logged to visdom
